In [130]:
import findspark
findspark.init('/home/user/spark-2.1.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('basics').getOrCreate()
df = spark.read.csv('Edudata.csv',header=True,inferSchema=True)

#df.show()
#newcol=df.withColumn('aaa',df['Higher_education_enrollment_rate']*1.0)
#newcol.show()
df.printSchema()
print("before",df.count())
#Heer_col = df.select('aaa')
#Heer_col.show()
df2=df.filter("Higher_education_enrollment_rate > 0")
df3=df2.filter("Gross_enrolment_ratio > 0")
#dropped = df.na.drop(subset="Higher_education_enrollment_rate")
#dropped.show()
#df3.show()
#print("drop count",dropped.count())
print("after",df2.count())
print("after2",df3.count())
from pyspark.ml.regression import LinearRegression
datas=df3.select(df3.Higher_education_enrollment_rate.cast('float'),df3.Gross_enrolment_ratio.cast('float'))
datas.show()
from pyspark.ml.linalg import Vectors 
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler( inputCols=["Gross_enrolment_ratio"], outputCol="features")
output = assembler.transform(datas)
label_features=output.select("features","Higher_education_enrollment_rate").toDF("features","label")
label_features.show(truncate=False)

lr = LinearRegression(featuresCol='features',labelCol='label')
lrModel = lr.fit(label_features)
print("Coefficients: %s" % str(lrModel.coefficients))
print("Intercept: %s" % str(lrModel.intercept))
trainingSummary = lrModel.summary
print("numIterations: %d" % trainingSummary.totalIterations)
print("objectiveHistory: %s" % str(trainingSummary.objectiveHistory))
trainingSummary.residuals.show()
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)










root
 |-- Time: integer (nullable = true)
 |-- Country_Name: string (nullable = true)
 |-- Per_capita_GDP_rate: string (nullable = true)
 |-- Per_capita_GDP: double (nullable = true)
 |-- Gross_domestic_productA_at_market_prices: decimal(14,0) (nullable = true)
 |-- Population_growth_rate: double (nullable = true)
 |-- Male_population: double (nullable = true)
 |-- Female_population: double (nullable = true)
 |-- Literacy_rate_male: double (nullable = true)
 |-- Literacy_rate_female: double (nullable = true)
 |-- Government_education_expenditure_as_a_percentage_of_GDP: double (nullable = true)
 |-- Government_expenditure_on_secondary_and_higher_vocational_education_as_a_percentage_of_GDP: double (nullable = true)
 |-- Total_population: integer (nullable = true)
 |-- Higher_education_enrollment: integer (nullable = true)
 |-- Higher_education_enrollment_rate: double (nullable = true)
 |-- Educational_Fund : string (nullable = true)
 |-- Enrollment_Rate: string (nullable = true)
 |-- Pri